In [152]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 999

In [153]:
train = pd.read_csv("mertrain.csv")
test = pd.read_csv("mertest.csv")

In [154]:
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(train[c].values) + list(test[c].values)) 
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))
        

In [155]:
y = train['y'].values
y_mean = np.mean(y)
id_test = test['ID'].values
X = train.drop(['y','ID'], axis=1).values
X_test= test.drop(['ID'], axis=1).values

In [157]:
#train_important=train[["X315","X314","X118"]]
#test_important=test[["X315","X314","X118"]]

In [158]:
#train_important.head()

,X315,X314,X118
0,0,0,1
1,0,0,1
2,0,0,0
3,0,0,0
4,0,0,0


In [159]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD


In [160]:
 n_comp = 320
pca = PCA(n_components=n_comp, random_state=420)
#pca2_results_train = pca.fit_transform(train.drop(["y"], axis=1))
#pca2_results_test = pca.transform(test)

In [161]:
y = train['y'].values
y_mean = np.mean(y)
id_test = test['ID'].values
X_test=test.values

In [162]:
df_pca_train=pd.DataFrame(pca2_results_train)
df_pca_test=pd.DataFrame(pca2_results_test)

In [163]:
train_pca_important= pd.concat([df_pca_train, train_important], axis=1, join='inner')
test_pca_important=pd.concat([df_pca_test, test_important], axis=1, join='inner')

In [166]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5, shuffle=True, random_state=0)
kf.get_n_splits(train_pca_important)

5

In [167]:
for train_index, test_index in kf.split(train_pca_important):
    X_train, y_train=train_pca_important.values[train_index], y[train_index]
    X_valid, y_valid=train_pca_important.values[test_index], y[test_index]

In [168]:
forest= RandomForestRegressor(n_estimators=100, random_state=0, max_depth=5)
forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

In [169]:
from sklearn.metrics import r2_score
print("R^2 valid: {: }".format(r2_score(y_valid, forest.predict(X_valid))))
print("R^2 train: {: }".format(r2_score(y_train, forest.predict(X_train))))

R^2 valid:  0.581491168803387
R^2 train:  0.6260073624572866


In [170]:
print(r2_score(y, forest.predict(train_pca_important)))

0.617771837042


In [171]:
y_pred=forest.predict(test_pca_important)

In [172]:
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = y_pred
sub.to_csv('forest-pca70-important3.csv', index=False)


In [173]:
import xgboost as xgb

# prepare dict of params for xgboost to run with
xgb_params = {
    'n_trees': 500, 
    'eta': 0.005,
    'max_depth': 4,
    'subsample': 0.95,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}

# form DMatrices for Xgboost training
dtrain = xgb.DMatrix(X_train, y_train)
dvalid=xgb.DMatrix(X_valid, y_valid)
dtest = xgb.DMatrix(test_pca_important.values)

# xgboost, cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   num_boost_round=500, # increase to have better results (~700)
                   early_stopping_rounds=50,
                   verbose_eval=50, 
                   show_stdv=False
                  )

num_boost_rounds = len(cv_result)
print(num_boost_rounds)

# train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

[0]	train-rmse:12.7346	test-rmse:12.7292
[50]	train-rmse:11.1706	test-rmse:11.2793
[100]	train-rmse:10.0807	test-rmse:10.2974
[150]	train-rmse:9.33195	test-rmse:9.65347
[200]	train-rmse:8.82148	test-rmse:9.24336
[250]	train-rmse:8.47398	test-rmse:8.98572
[300]	train-rmse:8.22737	test-rmse:8.8289
[350]	train-rmse:8.01671	test-rmse:8.73754
[400]	train-rmse:7.81701	test-rmse:8.6877
[450]	train-rmse:7.64432	test-rmse:8.66378
500


In [174]:
from sklearn.metrics import r2_score
print("R^2 valid: {: }".format(r2_score(y_valid, model.predict(dvalid))))
print("R^2 train: {: }".format(r2_score(y_train, model.predict(dtrain))))

R^2 valid:  0.579440183524708
R^2 train:  0.6280580586349003


In [175]:
y_pred = model.predict(dtest)

In [176]:
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = y_pred
sub.to_csv('xgboost-pca70-important3.csv', index=False)

In [177]:
from sklearn.linear_model import Ridge

In [178]:
reg=Ridge(alpha=1.0)

In [179]:
ridge=reg.fit(X_train,y_train)

In [180]:
from sklearn.metrics import r2_score
print("R^2 valid: {: }".format(r2_score(y_valid, ridge.predict(X_valid))))
print("R^2 train: {: }".format(r2_score(y_train, ridge.predict(X_train))))

R^2 valid:  0.5755723140582918
R^2 train:  0.5586120260411072


In [181]:
y_pred=ridge.predict(test_pca_important)

In [182]:
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = y_pred
sub.to_csv('ridge-pca70-important3.csv', index=False)


In [183]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

In [184]:
scaler=MaxAbsScaler()

In [185]:
scaler.fit(X_train)

MaxAbsScaler(copy=True)

In [186]:
X_train_scaled=scaler.transform(X_train)
X_valid_scaled=scaler.transform(X_valid)
X_test_scaled=scaler.transform(test_pca_important)

In [187]:
from sklearn.svm import SVR

In [188]:
model=SVR(gamma='auto', C=100)

In [189]:
model.fit(X_train_scaled, y_train)


SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [190]:
from sklearn.metrics import r2_score
print("R^2 train:{}".format(r2_score(y_train, model.predict(X_train_scaled))))
print("R^2 valid:{}".format(r2_score(y_valid, model.predict(X_valid_scaled))))

R^2 train:0.5598328517867093
R^2 valid:0.559349624253275


In [191]:
y_pred= model.predict(X_test_scaled)

In [192]:
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = y_pred
sub.to_csv('svr-pca70-important3.csv', index=False)
